## Calculation of the SDEB (Square-root Daily Exceedance and Bias) objective function

In [35]:
# This objective function combines multiple terms to try to simultaneously achieve good fits in
#    - overall bias
#    - daily flows
#    - daily exceedance statistics

# It is particularly well-suited for Rainfall-Runoff model calibration. 
# One of the nice things about it is that it doesn't have any free parameters.

In [36]:
# Generate some dummy data for this example
import math
import numpy as np
import random
observed_flow = []
modelled_flow = []
npoints = 1000
for i in range(npoints):    
    #add some random observed value
    observed_flow.append(100 * random.random())
    #but every now and then replace it with NAN so our data has some missing values
    if (random.random() < 0.05):        
        observed_flow[i] = float('nan')
    #add some random modelled value
    modelled_flow.append(100 * random.random())

In [37]:
print(len(modelled_flow))
print(len(observed_flow))

1000
1000


In [38]:
#the first thing to do is filter out all the missing data from both datasets
observed_nomissing = []
modelled_nomissing = []
npoints_nomissing = 0
for i in range(npoints):
    if (not math.isnan(observed_flow[i])):
        observed_nomissing.append(observed_flow[i])
        modelled_nomissing.append(modelled_flow[i])
        npoints_nomissing = npoints_nomissing + 1
        
npoints_nomissing

944

In [39]:
#now calculate the bias term
total_obs = 0
total_mod = 0
for i in range(npoints_nomissing):
    total_obs += observed_nomissing[i]
    total_mod += modelled_nomissing[i]
        
bias_term = 1 + abs((total_mod - total_obs)/total_obs)
bias_term

1.052727095671755

In [40]:
#now calculate the daily term
daily_term = 0
for i in range(npoints_nomissing):
    temp = math.sqrt(observed_nomissing[i]) - math.sqrt(modelled_nomissing[i])
    daily_term = daily_term + temp * temp
    
daily_term

10853.748540582075

In [47]:
#now sort the data and calculate the exceedance term
#the form is similar to teh daily term
sorted_observed_nomissing = observed_nomissing[:]
sorted_modelled_nomissing = modelled_nomissing[:]
sorted_observed_nomissing.sort()
sorted_modelled_nomissing.sort()
exceedance_term = 0
for i in range(npoints_nomissing):
    temp = math.sqrt(sorted_observed_nomissing[i]) - math.sqrt(sorted_modelled_nomissing[i])
    exceedance_term = exceedance_term + temp * temp
    
exceedance_term

63.46312316981347

In [49]:
#and finally combine the terms in this special way to get the overall objective function value
#That value is what you want to minimize by optimization.
#This is what Rob calls the superobjective function.
SDEB = (0.1 * daily_term + 0.9 * exceedance_term) * bias_term
SDEB

1202.7319322309866

In [ ]:
#We did play with an additional term to try to force the baseflow fraction to a target value. 
#That the the purpose of the GAGA program. Then it is effectively modified like this:
#SDEB = (0.1 * daily_term + 0.9 * exceedance_term) * bias_term * baseflow_term
#but I would start without the baseflow stuff and see how you get on.